In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

# Import the requests library.
import requests

# Import gmaps
import gmaps

# Import the API keys.
from config import weather_api_key
from config import g_key

# Import the datetime module from the datetime library.
from datetime import datetime

In [2]:
WeatherPy_Vacation_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
WeatherPy_Vacation_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (1hr),Snow (1hr)
0,0,Albany,US,2020-05-11 00:13:00,42.60,-73.97,54.0,51,98,4.94,overcast clouds,0.0,0.0
1,1,Saint Austell,GB,2020-05-11 00:15:04,50.34,-4.77,46.4,61,84,13.87,broken clouds,0.0,0.0
2,2,Punta Arenas,CL,2020-05-11 00:15:04,-53.15,-70.92,42.8,87,40,8.05,scattered clouds,0.0,0.0
3,3,Tasiilaq,GL,2020-05-11 00:15:05,65.61,-37.64,28.4,92,17,11.41,few clouds,0.0,0.0
4,4,Teguise,ES,2020-05-11 00:15:05,29.06,-13.56,68.0,68,20,12.75,few clouds,0.0,0.0


In [3]:
# Ask the customer to add a minimum and maximum temperature value and rain/snow preference
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain = str(input("Do you want it to be raining? (yes/no) "))
snow = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [4]:
# Filter the dataset to find the cities that fit the criteria
if rain == "yes":
    if snow == "yes":
        pref_cities_df = WeatherPy_Vacation_df.loc[(WeatherPy_Vacation_df["Max Temp"] <= max_temp) & \
                                                   (WeatherPy_Vacation_df["Max Temp"] >= min_temp) & \
                                                   (WeatherPy_Vacation_df["Rain (1hr)"] > 0.0) & \
                                                   (WeatherPy_Vacation_df["Snow (1hr)"] > 0.0)].dropna()
    else:
        pref_cities_df = WeatherPy_Vacation_df.loc[(WeatherPy_Vacation_df["Max Temp"] <= max_temp) & \
                                                   (WeatherPy_Vacation_df["Max Temp"] >= min_temp) & \
                                                   (WeatherPy_Vacation_df["Rain (1hr)"] > 0.0) & \
                                                   (WeatherPy_Vacation_df["Snow (1hr)"] == 0.0)].dropna()
else:
    if snow == "yes":
        pref_cities_df = WeatherPy_Vacation_df.loc[(WeatherPy_Vacation_df["Max Temp"] <= max_temp) & \
                                                   (WeatherPy_Vacation_df["Max Temp"] >= min_temp) & \
                                                   (WeatherPy_Vacation_df["Rain (1hr)"] == 0.0) & \
                                                   (WeatherPy_Vacation_df["Snow (1hr)"] > 0.0)].dropna()
    else:
        pref_cities_df = WeatherPy_Vacation_df.loc[(WeatherPy_Vacation_df["Max Temp"] <= max_temp) & \
                                                   (WeatherPy_Vacation_df["Max Temp"] >= min_temp) & \
                                                   (WeatherPy_Vacation_df["Rain (1hr)"] == 0.0) & \
                                                   (WeatherPy_Vacation_df["Snow (1hr)"] == 0.0)].dropna()
        
    

pref_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (1hr),Snow (1hr)
0,0,Albany,US,2020-05-11 00:13:00,42.60,-73.97,54.00,51,98,4.94,overcast clouds,0.0,0.0
4,4,Teguise,ES,2020-05-11 00:15:05,29.06,-13.56,68.00,68,20,12.75,few clouds,0.0,0.0
5,5,Castro,BR,2020-05-11 00:15:05,-24.79,-50.01,55.17,89,0,4.32,clear sky,0.0,0.0
8,8,Belmonte,BR,2020-05-11 00:15:05,-15.86,-38.88,69.80,77,20,4.70,few clouds,0.0,0.0
10,10,Morro Bay,US,2020-05-11 00:15:05,35.37,-120.85,70.00,77,81,7.00,broken clouds,0.0,0.0
11,11,Kamaishi,JP,2020-05-11 00:15:05,39.27,141.88,64.40,63,40,10.29,shower rain,0.0,0.0
12,12,Tarakan,ID,2020-05-11 00:15:05,3.30,117.63,81.46,81,60,6.06,broken clouds,0.0,0.0
13,13,Kropotkin,RU,2020-05-11 00:15:05,45.44,40.58,50.00,85,66,13.02,broken clouds,0.0,0.0
14,14,Murwillumbah,AU,2020-05-11 00:15:05,-28.32,153.40,70.00,52,10,13.87,clear sky,0.0,0.0
15,15,Hermanus,ZA,2020-05-11 00:15:06,-34.42,19.23,54.00,94,0,3.65,clear sky,0.0,0.0


In [5]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_WeatherPy_df = pref_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_WeatherPy_df["Hotel Name"] = ""
hotel_WeatherPy_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,54.00,overcast clouds,42.60,-73.97,
4,Teguise,ES,68.00,few clouds,29.06,-13.56,
5,Castro,BR,55.17,clear sky,-24.79,-50.01,
8,Belmonte,BR,69.80,few clouds,-15.86,-38.88,
10,Morro Bay,US,70.00,broken clouds,35.37,-120.85,


In [6]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_WeatherPy_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_WeatherPy_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_WeatherPy_df.head()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,54.00,overcast clouds,42.60,-73.97,
4,Teguise,ES,68.00,few clouds,29.06,-13.56,B&B La Casona de Nazaret
5,Castro,BR,55.17,clear sky,-24.79,-50.01,CHACARA BAILLY
8,Belmonte,BR,69.80,few clouds,-15.86,-38.88,Pousada Monte Carmelo O Pão
10,Morro Bay,US,70.00,broken clouds,35.37,-120.85,Inn at Morro Bay


In [9]:
# Create the output file (CSV).
output_data_file_2 = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_WeatherPy_df.to_csv(output_data_file_2, index_label="City_ID")

In [7]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_WeatherPy_df.iterrows()]

In [8]:
# Add a map of cities for the vacation spots.
locations = hotel_WeatherPy_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))